In [1]:
# %load_ext autoreload
# %autoreload 2
import models, utils
import orca
import dask.dataframe as dd
from __future__ import division
import csv
import numpy as np
import pandas as pd
import logging
#import greenroof


In [21]:
greenroof.NoInterventionGreenRoof()
greenroof.SanFranGreenRoof()
greenroof.ChicagoGreenRoof()
greenroof.TorontoGreenRoof()
print("success")

success


In [24]:
buildings = orca.get_table('buildings')
buildings.to_frame()
print(orca.list_columns())
mybuild = orca.orca.DataFrameWrapper.get_column(buildings, 'unit_sqft')
mybuild.to_frame()
mybuild.sort_index(inplace=True)
mybuild.to_frame()
height = orca.orca.DataFrameWrapper.get_column(buildings, 'stories')
height.sort_index(inplace=True)
height.to_frame()
purpose = orca.orca.DataFrameWrapper.get_column(buildings, 'general_type')
purpose.sort_index(inplace=True)
purpose.to_frame()
dataset = pd.concat([height, mybuild, purpose], axis=1)
dataset
#dataset.to_csv('data.csv') #uncomment to place is csv file and fix formatting
#height of building approx = 11.5 feet * stories

Filling column residential_sales_price with value 0 (14196 values)
Filling column building_type_id with value 2.0 (0 values)
Filling column residential_units with value 0 (0 values)
Filling column year_built with value 1927.0 (3116 values)
Filling column non_residential_sqft with value 0 (1341 values)
Filling column non_residential_rent with value 0 (142400 values)
Filling column job_category with value service (331 values)
[('buildings', 'vacant_residential_units'), ('buildings', 'sqft_per_job'), ('zones_prices', 'zone_id'), ('zones', 'hhsize'), ('zones', 'sum_residential_units'), ('parcels', 'total_job_spaces'), ('zones_prices', 'retail'), ('parcels', 'max_height'), ('zones', 'ave_unit_sqft'), ('parcels', 'land_cost'), ('jobs', 'zone_id'), ('buildings', 'vacant_job_spaces'), ('buildings', 'general_type'), ('zones_prices', 'residential'), ('buildings', 'unit_lot_size'), ('zones', 'zone_id'), ('households', 'zone_id'), ('parcels', 'parcel_size'), ('buildings', 'job_spaces'), ('zones', 

,stories,0,building_type_id
building_id,,,
1,2.0,1.232000e+03,Residential
2,1.0,1.553500e+03,Residential
3,2.0,2.365000e+03,Residential
4,2.0,1.348000e+03,Residential
5,2.0,2.095000e+03,Residential
6,1.0,7.630000e+02,Residential
7,2.0,1.125000e+03,Residential
8,2.0,1.125000e+03,Residential
9,1.0,1.376000e+03,Residential


In [89]:
buildings = orca.get_table('buildings')
buildings.to_frame()
buildings['general_type'][1]

'Residential'

In [46]:
#TO DO: Gradually Transfer this alg to data cell below

data = pd.read_csv('data.csv')
roofinstalled_dict = {}
roof_cost_at_20yrs_dict = {}
green_roof_benefit_dict = {}
green_roof_cost_dict = {}
green_roof_net_benefit_dict = {}     
building_size_dict = {}
#greenroof.NoInterventionGreenRoof()
green_cost_per_sqft = 12

for index in range(-1, 1000, 1):
    index = index + 1
    building_size = (data['sq_ft'][index])/(data['stories'][index])
    building_size_dict.update({index : building_size})
    #Initialization of assumed and derived variables
    base_green_cost_per_sqft = 12
    green_roof_min = 0
    conventional_cost_per_sqft = green_cost_per_sqft/2
    green_roof_benefits = 0
    building_age = 40
    annual_benefit_per_area = .23
    annual_maintenance_costs_per_area = 0.02 
    min_green_area = green_roof_min * building_size
    min_green_cost = green_roof_min * green_cost_per_sqft
    best_green_cost = min_green_cost #default value
    min_green_benefits= green_roof_min*building_size*annual_benefit_per_area*building_age
    best_green_net_benefit = min_green_benefits #default value
    best_size = green_roof_min
    best_benefit = min_green_benefits - min_green_cost
    best_conv = (building_size*(1-green_roof_min)*conventional_cost_per_sqft) #Avoid capital costs
    roofinstalled_dict.update({index : 1}) #this green roof variable can be stored in the final



    #Calculation of Output variables

    #If the building is large enough to allow bulk discounting to begin (happens in most cases)
    if (building_size > 500):
        green_cost_per_sqft = 23.907*(base_green_cost_per_sqft**(-0.078))

    #for current_size in my_range(green_roof_min, 1, 0.1):
    #    roof_cost_at_20yrs = (building_size*(1-current_size)*conventional_cost_per_sqft) #Avoid capital costs
    #    green_roof_cost = current_size * building_size * green_cost_per_sqft + (building_age * annual_maintenance_costs_per_area * building_size * current_size)
    #    green_roof_benefits = building_age * annual_benefit_per_area * building_size * current_size + roof_cost_at_20yrs
    #    if(green_roof_benefits - green_roof_cost > best_benefit):
    #        best_size = current_size
    #        best_green_cost = green_roof_cost
    #        best_green_benefits = green_roof_benefits
    #        best__green_net_benefit = green_roof_benefits - green_roof_cost
    #        best_conv = roof_cost_at_20yrs

    if(best_benefit > 0):
        roofinstalled_dict.update({index : 1}) #this green roof variable can be stored in the final
    else:
        roofinstalled_dict.update({index : 0})

    roof_cost_at_20yrs_dict.update({index : best_conv})
    green_roof_benefit_dict.update({index : best_benefit})
    green_roof_cost_dict.update({index : best_green_cost})
    green_roof_net_benefit_dict.update({index : best_green_net_benefit})
    index = index +1   

In [96]:
data['building_type_id'][2]

'Residential'

In [66]:
#WORK SHOULD BE DONE IN HERE
data = pd.read_csv('data.csv')
def my_range(start, end, step):
    while start <= end:
        yield start #keyword yield generates an interator
        start += step

building_size_dict = {}
roofins_dict = {}
green_roof_ben_dict = {}
green_roof_cost_dict = {}
roof_cost_at_20yrs_dict = {}
size_dict = {}

for index in range(-1, 2000, 1):
    index = index + 1
    best_ben = 0
    building_size = (data['sq_ft'][index])/(data['stories'][index])
    roof_cost_at_20yrs = building_size * 6
    building_size_dict.update({index : building_size})
    
    for size_ratio in my_range (0, 1.01, .01):
        green_roof_cost = 23.907*(12**(-0.078)) *size_ratio * building_size + (40*.21*building_size*size_ratio)
        green_roof_ben = .23 * size_ratio * building_size * 40 + (building_size * 6 * (1-size_ratio))
        if(green_roof_ben - green_roof_cost > best_ben):
            roof_ins = 1
            best_size = size_ratio * building_size
            size_dict.update({index : best_size})
            roofins_dict.update({index : roof_ins})
            green_roof_ben_dict.update({index : green_roof_ben})
            green_roof_cost_dict.update({index : green_roof_cost})
            roof_cost_at_20yrs = building_size * 6 * (1-size_ratio)            
            roof_cost_at_20yrs_dict.update({index : roof_cost_at_20yrs})

buildings = pd.DataFrame({"building size" : building_size_dict, "best_size" : size_dict, "roofins" : roofins_dict, "green_ben" : green_roof_ben_dict, "green_cost" : green_roof_cost_dict, "cost_20yr" : roof_cost_at_20yrs_dict, })
writer = pd.ExcelWriter('no_action.xlsx')
buildings.to_excel(writer,'no_action')
writer.save()

In [67]:
building_size_dict = {}
roofins_dict = {}
green_roof_ben_dict = {}
green_roof_cost_dict = {}
roof_cost_at_20yrs_dict = {}
size_dict = {}
for index in range(-1, 2000, 1):
    index = index + 1
    best_ben = 0
    building_size = (data['sq_ft'][index])/(data['stories'][index])
    roof_cost_at_20yrs = building_size * 6
    building_size_dict.update({index : building_size})
    for size_ratio in my_range (0, 1.01, .01):
        green_roof_cost = 23.907*(12**(-0.078)) *size_ratio * building_size + (40*.21*building_size*size_ratio)
        green_roof_ben = .23 * size_ratio * building_size * 40 + (building_size * 6 * (1-size_ratio)) + 0.05*building_size*size_ratio
        if(green_roof_ben - green_roof_cost > best_ben):
            roof_ins = 1
            best_size = size_ratio * building_size
            size_dict.update({index : best_size})
            roofins_dict.update({index : roof_ins})
            green_roof_ben_dict.update({index : green_roof_ben})
            green_roof_cost_dict.update({index : green_roof_cost})
            roof_cost_at_20yrs = building_size * 6 * (1-size_ratio)            
            roof_cost_at_20yrs_dict.update({index : roof_cost_at_20yrs})

buildings = pd.DataFrame({"building size" : building_size_dict, "best_size" : size_dict, "roofins" : roofins_dict, "green_ben" : green_roof_ben_dict, "green_cost" : green_roof_cost_dict, "cost_20yr" : roof_cost_at_20yrs_dict})
writer = pd.ExcelWriter('chicago.xlsx')
buildings.to_excel(writer,'chicago')
writer.save()

In [68]:
building_size_dict = {}
roofins_dict = {}
green_roof_ben_dict = {}
green_roof_cost_dict = {}
roof_cost_at_20yrs_dict = {}
size_dict = {}
mandated_dict = {}
for index in range(-1, 2000, 1):
    index = index + 1
    best_ben = 0
    building_size = (data['sq_ft'][index])/(data['stories'][index])
    roof_cost_at_20yrs = building_size * 6
    building_size_dict.update({index : building_size})
    
    if(data['building_type_id'][index] != 'Residential' and building_size < 15625):
        mandated_dict.update({index : 1})
        green_roof_cost = 23.907*(12**(-0.078)) *size_ratio * building_size + (40*.21*building_size*size_ratio)
        green_roof_ben = .23 * size_ratio * building_size * 40 + (building_size * 6 * (1-size_ratio))
        roof_ins = 1
        best_size = building_size
        size_dict.update({index : building_size})
        roofins_dict.update({index : roof_ins})
        green_roof_ben_dict.update({index : green_roof_ben})
        green_roof_cost_dict.update({index : green_roof_cost})
        roof_cost_at_20yrs_dict.update({index : 0})
        
    elif(data['building_type_id'][index] != 'Residential' and building_size >= 15625):
        mandated_dict.update({index : 1})
        green_roof_cost = 23.907*(12**(-0.078)) *size_ratio * building_size + (40*.21*building_size*size_ratio)
        green_roof_ben = .23 * size_ratio * building_size * 40 + (building_size * 6 * (1-size_ratio))
        roof_ins = 1
        best_size = 15625
        size_dict.update({index : 15625})
        roofins_dict.update({index : roof_ins})
        green_roof_ben_dict.update({index : green_roof_ben})
        green_roof_cost_dict.update({index : green_roof_cost})
        roof_cost_at_20yrs_dict.update({index : 0})
    
    else:
        mandated_dict.update({index : 0})
        for size_ratio in my_range (0, 1.01, .01):
            green_roof_cost = 23.907*(12**(-0.078)) *size_ratio * building_size + (40*.21*building_size*size_ratio)
            green_roof_ben = .23 * size_ratio * building_size * 40 + (building_size * 6 * (1-size_ratio))
            if(green_roof_ben - green_roof_cost > best_ben):
                roof_ins = 1
                best_size = size_ratio * building_size
                size_dict.update({index : best_size})
                roofins_dict.update({index : roof_ins})
                green_roof_ben_dict.update({index : green_roof_ben})
                green_roof_cost_dict.update({index : green_roof_cost})
                roof_cost_at_20yrs = building_size * 6 * (1-size_ratio)            
                roof_cost_at_20yrs_dict.update({index : roof_cost_at_20yrs})

buildings = pd.DataFrame({"building size" : building_size_dict, "best_size" : size_dict, "roofins" : roofins_dict, "green_ben" : green_roof_ben_dict, "green_cost" : green_roof_cost_dict, "cost_20yr" : roof_cost_at_20yrs_dict, "mandated" : mandated_dict})
writer = pd.ExcelWriter('san_fran.xlsx')
buildings.to_excel(writer,'san_fran')
writer.save()

In [69]:
building_size_dict = {}
roofins_dict = {}
green_roof_ben_dict = {}
green_roof_cost_dict = {}
roof_cost_at_20yrs_dict = {}
size_dict = {}
mandated_dict = {}
green_roof_min_dict = {}
for index in range(-1, 2000, 1):
    index = index + 1
    best_ben = 0
    building_size = (data['sq_ft'][index])/(data['stories'][index])
    roof_cost_at_20yrs = building_size * 6
    building_size_dict.update({index : building_size})
    #REVERT OFFICE #51 to 1330 sq.ft
    if(data['building_type_id'][index] != 'Residential' or ((data['building_type_id'][index] == 'Residential' and data['stories'][index] > 6))):
        mandated_dict.update({index : 1})
        if(building_size > 21525 and building_size <= 53800):
            green_roof_min = .2
        elif(building_size > 53800 and building_size <= 107625):
            green_roof_min = .3 
        elif(building_size > 107625 and building_size <= 161450):
            green_roof_min = .4 
        elif(building_size > 161450 and building_size <= 215275):
            green_roof_min = .5 
        elif (building_size > 215275):
            green_roof_min = .6
        else:
            green_roof_min = 0
              
        green_roof_min_dict.update( {index : green_roof_min})
        green_roof_cost = 23.907*(12**(-0.078)) * building_size + (40*.21*building_size*green_roof_min)
        green_roof_ben = 1.23 * building_size * 40 + (building_size * 6 * (1-green_roof_min))
        roof_ins = 1
        best_size = building_size * green_roof_min
        size_dict.update({index : best_size})
        roofins_dict.update({index : roof_ins})
        green_roof_ben_dict.update({index : green_roof_ben})
        green_roof_cost_dict.update({index : green_roof_cost})
        roof_cost_at_20yrs_dict.update({index : 0})

        for size_ratio in my_range (green_roof_min, 1.01, .01):
            green_roof_cost = 23.907*(12**(-0.078)) *size_ratio * building_size + (40*.21*building_size*size_ratio)
            green_roof_ben = .23 * size_ratio * building_size * 40 + (building_size * 6 * (1-size_ratio))
            if(green_roof_ben - green_roof_cost > best_ben):
                roof_ins = 1
                best_size = size_ratio * building_size
                size_dict.update({index : best_size})
                roofins_dict.update({index : roof_ins})
                green_roof_ben_dict.update({index : green_roof_ben})
                green_roof_cost_dict.update({index : green_roof_cost})
                roof_cost_at_20yrs = building_size * 6 * (1-size_ratio)            
                roof_cost_at_20yrs_dict.update({index : roof_cost_at_20yrs})

    else:
        mandated_dict.update({index : 0})
        green_roof_min_dict.update( {index : 0})
        for size_ratio in my_range (0, 1.01, .01):
            green_roof_cost = 23.907*(12**(-0.078)) *size_ratio * building_size + (40*.21*building_size*size_ratio)
            green_roof_ben = .23 * size_ratio * building_size * 40 + (building_size * 6 * (1-size_ratio))
            if(green_roof_ben - green_roof_cost > best_ben):
                roof_ins = 1
                best_size = size_ratio * building_size
                size_dict.update({index : best_size})
                roofins_dict.update({index : roof_ins})
                green_roof_ben_dict.update({index : green_roof_ben})
                green_roof_cost_dict.update({index : green_roof_cost})
                roof_cost_at_20yrs = building_size * 6 * (1-size_ratio)            
                roof_cost_at_20yrs_dict.update({index : roof_cost_at_20yrs})

buildings = pd.DataFrame({"building size" : building_size_dict, "best_size" : size_dict, "roofins" : roofins_dict, "green_ben" : green_roof_ben_dict, "green_cost" : green_roof_cost_dict, "cost_20yr" : roof_cost_at_20yrs_dict, "mandated" : mandated_dict, "green_min" : green_roof_min_dict})
writer = pd.ExcelWriter('toronto.xlsx')
buildings.to_excel(writer,'toronto')
writer.save()


In [ ]:
parcels = orca.get_table('parcels')
parcels.to_frame()

In [ ]:
buildings = orca.get_table('buildings')
buildings.to_frame()

In [ ]:
zones = orca.get_table('zones')
zones.to_frame()

In [2]:
orca.run([
    "rsh_simulate",              # residential sales hedonic
    "nrh_simulate",              # non-residential rent hedonic

    "households_relocation",     # households relocation model
    "hlcm_simulate",            # households location choice
    "households_transition",     # households transition

    "jobs_relocation",           # jobs relocation model
    "elcm_simulate",             # employment location choice
    "jobs_transition",           # jobs transition

    "feasibility",               # compute development feasibility
    "residential_developer",     # build residential buildings
    "non_residential_developer", # build non-residential buildings
], iter_vars=[2010, 2011, 2012, 2013, 2014])

Running iteration 1 with iteration value 2010
Running step 'rsh_simulate'
Filling column residential_sales_price with value 0 (14196 values)
Filling column non_residential_rent with value 0 (142400 values)
Filling column non_residential_sqft with value 0 (1341 values)
Filling column residential_units with value 0 (0 values)
Filling column year_built with value 1927.0 (3116 values)
Filling column building_type_id with value 2.0 (0 values)
count    1.407800e+05
mean     4.271533e+03
std      3.744239e+05
min      1.204550e-04
25%      1.899947e+02
50%      3.179589e+02
75%      5.150183e+02
max      1.210341e+08
dtype: float64
Time to execute step 'rsh_simulate': 0.96 s
Running step 'nrh_simulate'
Filling column job_category with value service (331 values)
count    10803.000000
mean        20.729909
std          6.833292
min          6.258496
25%         17.431477
50%         20.908725
75%         25.077906
max         47.266957
dtype: float64
Time to execute step 'nrh_simulate': 0.83 s


C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\developer\sqftproforma.py:394: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ave_cost_sqft'][c.fars > c.max_industrial_height] = np.nan
C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\developer\sqftproforma.py:392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ave_cost_sqft'][c.fars > c.max_retail_height] = np.nan


Describe of the yearly rent by use
              retail    industrial         office    residential
count  152205.000000  94493.000000  152259.000000  153228.000000
mean       23.867557      8.650493      21.274562      16.511098
std         3.553082      1.641869       2.979941       5.507942
min        18.000000      6.000000      16.000000       0.500000
25%              NaN           NaN            NaN            NaN
50%              NaN           NaN            NaN            NaN
75%              NaN           NaN            NaN            NaN
max        35.000000     14.500000      31.000000      72.800000
Computing feasibility for form mixedresidential


C:\Users\Harrison Freund\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


Computing feasibility for form retail
Computing feasibility for form industrial
Computing feasibility for form office
Computing feasibility for form residential
Computing feasibility for form mixedoffice
Time to execute step 'feasibility': 3.45 s
Running step 'residential_developer'
Number of agents: 362,867
Number of agent spaces: 374,133
Current vacancy = 0.03
Target vacancy = 0.15, target of new units = 52,769
3,838 feasible buildings before running developer
Sum of net units that are profitable: 33,312
WARNING THERE WERE NOT ENOUGH PROFITABLE UNITS TO MATCH DEMAND
Adding 1,695 buildings with 38,694 residential_units
2,143 feasible buildings after running developer
Time to execute step 'residential_developer': 0.32 s
Running step 'non_residential_developer'
Number of agents: 236,711
Number of agent spaces: 642,004
Current vacancy = 0.63
Target vacancy = 0.15, target of new units = 0
2,143 feasible buildings before running developer
Sum of net units that are profitable: 108,914
Addin

In [77]:
import numpy
a = [1, 1, 1 ,1, 1]
ar = numpy.array(a)
ar = ar+2
print (ar + 2)

[5 5 5 5 5]
